In [1]:
import numpy as np
import pandas as pd

## Helper functions: attendance and calculate the staff

In [2]:
## helper functions
def asistanceGenerator(min_asistance, max_asistance, day): #   generate asistance min_asistance = 5    max_asistance = 161
    if day == 'monday':
        max_asistance -= 120
    elif day == 'tuesday':
        max_asistance -= 85
    elif day == 'wednesday':
        max_asistance -= 95
    elif day == 'thursday':
        max_asistance -= 20
    elif day == 'friday':
        max_asistance -= 10
        min_asistance = max_asistance - 50
    elif day == 'saturday':
        max_asistance -= 0
        min_asistance = max_asistance - 20
    elif day == 'sunday':
        min_asistance = max_asistance - 5
        max_asistance -= 0  

    children = int(np.random.randint(min_asistance,max_asistance) * 0.4)   ### pertange by movie
    adults = int(np.random.randint(min_asistance,max_asistance) * 0.45)
    seniors = int(np.random.randint(min_asistance,max_asistance) * 0.15)
    return (children, adults, seniors, children + adults + seniors)

# 100%-70 snacks for all screen: 100-70% - 3, 69-40% - 2, 40-0% - 1. Same for tickets saler
# 100%-70 cleaning - 3, technician 1,  
# 69%-40 cleaning - 2, technician 1,
# 40%-0 cleaning - 1, technician 1,
def calculateStaff(perc):
    if perc <= 100 and perc > 70:
        cleaning_dependant = 4
        theater_technician = 1
        return cleaning_dependant + theater_technician
    if perc <= 70 and perc > 50:
        cleaning_dependant = 3
        theater_technician = 1
        return cleaning_dependant + theater_technician
    elif perc <= 50 and perc > 30:
        cleaning_dependant = 2
        theater_technician = 1
        return cleaning_dependant + theater_technician
    elif perc <= 30 and perc > 0:
        cleaning_dependant = 1
        theater_technician = 1
        return cleaning_dependant + theater_technician
    elif perc == 0:
        cleaning_dependant = 1
        return cleaning_dependant  


## Structure that supports the attendance by screens and hours

In [3]:
week_list = []

week_dict = {
    'monday':{
        'screens':{
            '1': {            
                '7':[],
                '9':[]
            },
            '2': {            
                '7':[],
                '9':[]
            },
        }
    }, 
    'tuesday':{
        'screens':{
            '1': {            
                '7':[],
                '9':[]
            },
            '2': {            
                '7':[],
                '9':[]
            },
        }
    },
    'wednesday':{
        'screens':{
            '1': {            
                '7':[],
                '9':[]
            },
            '2': {            
                '7':[],
                '9':[]
            },
        }
    },
    'thursday':{
        'screens':{
            '1': {            
                '7':[],
                '9':[]
            },
            '2': {            
                '7':[],
                '9':[]
            },
        }
    }, 
    'friday':{
        'screens':{
            '1': {            
                '7':[],
                '9':[]
            },
            '2': {            
                '7':[],
                '9':[]
            },
        }
    },
    'saturday':{
        'screens':{
            '1': {            
                '5':[],     
                '7':[],
                '9':[]
            },
            '2': {            
                '5':[],     
                '7':[],
                '9':[]
            },
        }
    },
    'sunday':{
        'screens':{
            '1': {            
                '3':[],
                '5':[],     
                '7':[],
                '9':[]
            },
            '2': {       
                '3':[],
                '5':[],     
                '7':[],
                '9':[]
            },
            '3': {       
                '3':[],
                '5':[],     
                '7':[],
                '9':[]
            },
        }
    }
}

summary_weeks = []

In [4]:
def theatherSimulation(num_simul=1):
    min_asistance = 5
    max_capacity = 161
    for simulations in range(0, num_simul):
        for day, day_content in week_dict.items():
            for i in week_dict[day]['screens']:
                for j in week_dict[day]['screens'][i]:
                    df_asistance = pd.DataFrame(np.zeros((8,1),dtype=int), columns=['Movie'])
                    df_asistance.index = ['Asistance_Adult', 'Asistance_Children', 'Asistance_Senior', 'Total_Asistance','Sales_Adult', 'Sales_Children', 
                                        'Sales_Senior', 'Total_Sales']
                    asis = asistanceGenerator(min_asistance, max_capacity, day)
                    df_asistance.iloc[:3,0] = asis[0:3]
                    df_asistance.loc['Total_Asistance'] = asis[3]

                    ## ticket sales calculation: adults 6,34 , children: 4.23, seniors: 5
                    df_asistance.loc['Sales_Adult'] = df_asistance.loc['Asistance_Adult'] * 6.34
                    df_asistance.loc['Sales_Children'] = df_asistance.loc['Asistance_Children'] * 4.23
                    df_asistance.loc['Sales_Senior'] = df_asistance.loc['Asistance_Senior'] * 5
                    if day == 'Friday':
                        df_asistance.loc['Sales_Senior'] = df_asistance.loc['Asistance_Senior'] * (5 - (5 * 0.1))
                    df_asistance.iloc[7,0] = sum(df_asistance.iloc[4:7,0])                                  # calculate total sales

                    week_dict[day]['screens'][i][j].append(df_asistance)  #  add asistance to week

                    df_cost = pd.DataFrame(np.zeros((6,1),dtype=int), columns=['Movie']) 
                    df_cost.index = ['Total_Staff', 'Property_Rent', 'Insurance_Cost', 'Employee_Cost', 'Manager_Cost', 'Total_Cost']

                    # calculate staff: capacity per screen:
                    perc = (df_asistance.iloc[3,0] / max_capacity) * 100
                    df_cost.iloc[0,0] = calculateStaff(perc)

                    # calculate employee cost: staff
                    staff_salary = 19 ## salary per day (minimum)
                    df_cost.iloc[3,0] = df_cost.iloc[0,0] * staff_salary

                    # other expenses
                    manager_salary = 35
                    rent_one_screen = 10
                    insurance_one_screen = 5
                    df_cost.loc['Manager_Cost'] = manager_salary
                    df_cost.loc['Property_Rent'] = rent_one_screen
                    df_cost.loc['Insurance_Cost'] = insurance_one_screen
                    df_cost.loc['Total_Cost'] = sum(df_cost.iloc[1:5,0])
                    week_dict[day]['screens'][i][j].append(df_cost) # add cost to week

            week_list.append(week_dict)
        #print(week_dict)
        summary_weeks.append(week_dict)
        

In [88]:
# num_simul = 1
# theatherSimulation(num_simul)

In [5]:
### print whole report asistance and sales
def whole_report_sales():
    for i in summary_weeks:
        print("\n****** Summary all days ******")
        for k,v in i.items():
            print("Day  ",k.capitalize())
            for ke,va in v['screens'].items():
                print(f"-\tScreen {ke}")
                for ho,values in va.items():
                    print(f"\t\tHours {ho}")
                    print(values[0])
                print()
            print("\n")

def whole_report_costs():
    for i in summary_weeks:
        print("\n****** Summary all days ******")
        for k,v in i.items():
            print("Day  ",k.capitalize())
            for ke,va in v['screens'].items():
                print(f"\t- Screen {ke}")
                for ho,values in va.items():
                    print(f"\t--- Hour {ho}")
                    print(values[1])
                print()
            print("\n")

#whole_report_sales()
#whole_report_costs()

In [6]:
### get days sales

def get_sales_per_day():
    all_sales_per_day = []
    max_day_sales = {'day': "", "amount": 0}

    for i in summary_weeks:
        print("\n****** Summary sales all days ******")
        for k,v in i.items():
            sumdaysales = 0
            for ke,va in v['screens'].items():
                for ho,values in va.items():
                    sumdaysales += int(values[0].loc['Total_Sales'])
            print(f"Sales of {k.capitalize()} is {sumdaysales}")
            all_sales_per_day.append(sumdaysales)


def get_cost_per_day():
    all_sales_per_day = []
    max_day_sales = {'day': "", "amount": 0}

    for i in summary_weeks:
        print("\n****** Summary cost all days ******")
        for k,v in i.items():
            sumdaysales = 0
            for ke,va in v['screens'].items():
                for ho,values in va.items():
                    sumdaysales += int(values[1].loc['Total_Cost'])
            print(f"Cost of {k.capitalize()} is {sumdaysales}")
            all_sales_per_day.append(sumdaysales)

#get_sales_per_day()
#get_cost_per_day()


In [7]:
### get max day sales
def get_max_sales_day():
    max_day_sales = {'day': "", "amount": 0}
    for i in summary_weeks:
        #print("\n****** Summary all days ******")
        for k,v in i.items():
            sumdaysales = 0
            for ke,va in v['screens'].items():
                for ho,values in va.items():
                    sumdaysales += int(values[0].loc['Total_Sales'])
            if sumdaysales > max_day_sales['amount']:
                max_day_sales['amount'] = sumdaysales
                max_day_sales['day'] = k.capitalize()
    return max_day_sales


### get max day cost
def get_max_cost_day():
    max_day_cost = {'day': "", "amount": 0}
    for i in summary_weeks:
        for k,v in i.items():
            sumdaycost = 0
            for ke,va in v['screens'].items():
                for ho,values in va.items():
                    sumdaycost += int(values[1].loc['Total_Cost'])
            if sumdaycost > max_day_cost['amount']:
                max_day_cost['amount'] = sumdaycost
                max_day_cost['day'] = k.capitalize()
    return max_day_cost


#print(get_max_sales_day())
#print(get_max_cost_day())

In [8]:
def optionsMenu():
    print("************************ Theater simulator ************************")
    print("\nMenu")
    print("1. Start simulation (1)")
    print("2. Get sales report for all days (2)")
    print("3. Get cost report for all days (3)")
    print("4. Get day with highest sales (4)")
    print("5. Get day with highest cost (5)")
    print("6. Report sales for all days (6)")
    print("7. Report cost for all days (7)")
    print("8 for exit")

def menu():
    optionsMenu()
    getoption = input("\nType your option>")
    
    if int(getoption) == 1:
        print("\nStarting simulation")
        num_simul = int(input("\nType total amount of experiments>"))
        theatherSimulation(num_simul)
        menu()
    elif int(getoption) == 2:
        print("\nSales report per day")
        get_sales_per_day()
        menu()
    elif int(getoption) == 3:
        print("\nCost report per day")
        get_cost_per_day()
        menu()
    elif int(getoption) == 4:
        print("\nDay with max sales")
        print(get_max_sales_day())
        menu()
    elif int(getoption) == 5:
        print("\nDay with max cost")
        print(get_max_cost_day())
        menu()
    elif int(getoption) == 6:
        print("\nSales report all days")
        whole_report_sales()
        menu()
    elif int(getoption) == 7:
        print("\nCost report all days")
        whole_report_costs()
        menu()
    elif int(getoption) == 8:
        #print("Bye bye")
        quit()
        #sys.exit("\nBye bye")
    else:
        print("\nPlease type an options from the menu")
        menu()


menu()



************************ Theater simulator ************************

Menu
1. Start simulation (1)
2. Get sales report for all days (2)
3. Get cost report for all days (3)
4. Get day with highest sales (4)
5. Get day with highest cost (5)
6. Report sales for all days (6)
7. Report cost for all days (7)
8 for exit

Starting simulation
************************ Theater simulator ************************

Menu
1. Start simulation (1)
2. Get sales report for all days (2)
3. Get cost report for all days (3)
4. Get day with highest sales (4)
5. Get day with highest cost (5)
6. Report sales for all days (6)
7. Report cost for all days (7)
8 for exit

Sales report all days

****** Summary all days ******
Day   Monday
-	Screen 1
		Hours 7
                     Movie
Asistance_Adult       8.00
Asistance_Children   11.00
Asistance_Senior      2.00
Total_Asistance      21.00
Sales_Adult          50.72
Sales_Children       46.53
Sales_Senior         10.00
Total_Sales         107.25
		Hours 9
         